### Alpha Vantage 일별 주가 데이터 수집
  - 기능: 다중 종목의 일별 주가 데이터 수집
  - 대상종목: IBM, MSFT
  - 수집기간: 최근 100일 (compact 모드)
  - 데이터소스: Alpha Vantage API
  - 주요내용:
    - 일별 시가, 고가, 저가, 종가, 거래량 포함
    - JSON 응답을 DataFrame으로 변환
    - CSV 파일로 저장

In [6]:

import os
import pandas as pd
import requests

# API_KEY = os.environ.get('ALPHAVANTAGE_API_KEY')
API_KEY = 'OACKEXPDGZWGF7KE'
BASE = 'https://www.alphavantage.co/query'

symbols = ['IBM', 'MSFT']
params_template = {
    'function': 'TIME_SERIES_DAILY',   #  TIME_SERIES_DAILY_ADJUSTED 프리미엄 기능
    'outputsize': 'compact',
    'datatype': 'json'
}

os.makedirs('data/collected', exist_ok=True)


In [7]:
frames = []
if API_KEY:
    for sym in symbols:
        try:
            params = params_template | {'symbol': sym, 'apikey': API_KEY}
            r = requests.get(BASE, params=params, timeout=30)
            r.raise_for_status()
            js = r.json()
            ts = js.get('Time Series (Daily)', {})
            if not ts:
                continue
            df = pd.DataFrame(ts).T.reset_index().rename(columns={
                'index': 'date',
                '1. open': 'open',
                '2. high': 'high',
                '3. low': 'low',
                '4. close': 'close',
                '5. volume': 'volume'
            })
            df['symbol'] = sym
            frames.append(df)
        except Exception:
            continue
out_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


In [8]:
out_df.head()


date      open      high       low     close   volume symbol
0  2025-09-19  266.0500  267.8700  263.6400  266.4000  9858112    IBM
1  2025-09-18  258.8600  265.2300  256.8004  265.0000  4988421    IBM
2  2025-09-17  257.4950  260.9644  257.0100  259.0800  3974785    IBM
3  2025-09-16  256.2600  258.0000  254.4100  257.5200  2719918    IBM
4  2025-09-15  254.0200  259.0500  254.0000  256.2400  4028365    IBM

In [9]:
out = 'data/collected/alpha_vantage_daily.csv'
if not out_df.empty:
    out_df.to_csv(out, index=False)


In [10]:
f"Saved: {out}" if not out_df.empty else "No data saved (missing API_KEY or empty data)"


'Saved: data/collected/alpha_vantage_daily.csv'